In [2]:
from pymongo import MongoClient
import pprint
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json
import time

Request the URL

In [3]:
url = 'https://www.goodreads.com/list/show/264.Books_That_Everyone_Should_Read_At_Least_Once?page=1'
r = requests.get(url)

In [4]:
r.status_code 

200

Save into MongoDB

In [5]:
client = MongoClient()
db = client.goodreads

In [6]:
page1 = db.must_read_books

In [8]:
page1.insert_one({'html': r.text})

Parse text to get data

In [10]:
soup = BeautifulSoup(r.text, 'html.parser')

This should give me everything that is in the entry for each book - only indexing the first one

In [11]:
soup.find_all('tr', '')[0]

<tr itemscope="" itemtype="http://schema.org/Book">
<td class="number" valign="top">1</td>
<td valign="top" width="5%">
<div class="u-anchorTarget" id="2657"></div>
<div class="js-tooltipTrigger tooltipTrigger" data-resource-id="2657" data-resource-type="Book">
<a href="/book/show/2657.To_Kill_a_Mockingbird" title="To Kill a Mockingbird">
<img alt="To Kill a Mockingbird" class="bookCover" itemprop="image" src="https://i.gr-assets.com/images/S/compressed.photo.goodreads.com/books/1553383690i/2657._SY75_.jpg"/>
</a> </div>
</td>
<td valign="top" width="100%">
<a class="bookTitle" href="/book/show/2657.To_Kill_a_Mockingbird" itemprop="url">
<span aria-level="4" itemprop="name" role="heading">To Kill a Mockingbird</span>
</a> <br>
<span class="by">by</span>
<span itemprop="author" itemscope="" itemtype="http://schema.org/Person">
<div class="authorName__container">
<a class="authorName" href="https://www.goodreads.com/author/show/1825.Harper_Lee" itemprop="url"><span itemprop="name">Harper

Identify the elements that I want

In [19]:
entries = soup.find_all('tr', '')[0]

In [63]:
entries.find('td', 'number').text # ranking
entries.find('a', 'bookTitle').text.strip() # title
entries.find('a', 'bookTitle')['href'] # link to book
entries.find('a', 'authorName').text.rstrip() # author
entries.find('span', 'minirating').text.strip() # rating
entries.find('span', 'smallText uitext').find('a').text # score
entries.find('span', 'smallText uitext').find_all('a')[1].text # votes


'/book/show/2657.To_Kill_a_Mockingbird'

Add front end of url to completed working link

In [66]:
# link_pt1 = 'https://www.goodreads.com'
# link_pt2 = entries.find('a', 'bookTitle')['href']
link_lst = ['https://www.goodreads.com', entries.find('a', 'bookTitle')['href']]
full_link = ''.join(link_lst)

'https://www.goodreads.com/book/show/2657.To_Kill_a_Mockingbird'

In [74]:
b_title = []
author = []
ranking = []
rating = []
score = []
votes = []
links_pt2 = []

In [75]:
for idx, book in enumerate(soup.find_all('tr', '')):
    b_title.append(book.find('a', 'bookTitle').text.strip())
    author.append(book.find('a', 'authorName').text.strip())
    ranking.append(book.find('td', 'number').text)
    rating.append(book.find('span', 'minirating').text.strip())
    score.append(book.find('span', 'smallText uitext').find('a').text)
    votes.append(book.find('span', 'smallText uitext').find_all('a')[1].text)
    links_pt2.append(book.find('a', 'bookTitle')['href'])


In [77]:
b_title

['To Kill a Mockingbird',
 "Harry Potter and the Sorcerer's Stone (Harry Potter, #1)",
 '1984',
 'Pride and Prejudice',
 'The Diary of a Young Girl',
 'Animal Farm',
 'The Little Prince',
 'The Great Gatsby',
 'The Catcher in the Rye',
 'The Lord of the Rings',
 'The Book Thief',
 'Jane Eyre',
 'The Chronicles of Narnia (Chronicles of Narnia, #1-7)',
 'Lord of the Flies',
 'Romeo and Juliet',
 'Harry Potter and the Deathly Hallows (Harry Potter, #7)',
 'The Kite Runner',
 'The Giver (The Giver, #1)',
 'The Giving Tree',
 "Charlotte's Web",
 'Little Women',
 'The Hunger Games (The Hunger Games, #1)',
 'The Hobbit, or There and Back Again',
 'Green Eggs and Ham',
 'Of Mice and Men',
 'Harry Potter and the Prisoner of Azkaban (Harry Potter, #3)',
 'Fahrenheit 451',
 "Alice's Adventures in Wonderland & Through the Looking-Glass",
 'Wuthering Heights',
 'Gone with the Wind',
 'Night  (The Night Trilogy, #1)',
 'The Picture of Dorian Gray',
 'Holy Bible: King James Version',
 'Harry Potter a

Add front end of url to each relative path

In [81]:
def combine_url_parts(lst):
    
    full_links = []
    for idx, link in enumerate(lst):
        link_lst = ['https://www.goodreads.com', lst[idx]]
        full_link = ''.join(link_lst)
        full_links.append(full_link)
    return full_links

In [82]:
# combine_url_parts(links_pt2)

['https://www.goodreads.com/book/show/2657.To_Kill_a_Mockingbird',
 'https://www.goodreads.com/book/show/3.Harry_Potter_and_the_Sorcerer_s_Stone',
 'https://www.goodreads.com/book/show/40961427-1984',
 'https://www.goodreads.com/book/show/1885.Pride_and_Prejudice',
 'https://www.goodreads.com/book/show/48855.The_Diary_of_a_Young_Girl',
 'https://www.goodreads.com/book/show/170448.Animal_Farm',
 'https://www.goodreads.com/book/show/157993.The_Little_Prince',
 'https://www.goodreads.com/book/show/4671.The_Great_Gatsby',
 'https://www.goodreads.com/book/show/5107.The_Catcher_in_the_Rye',
 'https://www.goodreads.com/book/show/33.The_Lord_of_the_Rings',
 'https://www.goodreads.com/book/show/19063.The_Book_Thief',
 'https://www.goodreads.com/book/show/10210.Jane_Eyre',
 'https://www.goodreads.com/book/show/11127.The_Chronicles_of_Narnia',
 'https://www.goodreads.com/book/show/7624.Lord_of_the_Flies',
 'https://www.goodreads.com/book/show/18135.Romeo_and_Juliet',
 'https://www.goodreads.com/b